In [ ]:
from orbit_generation.experiment import generate_parameter_sets, paralelize_notebook_experiment

Parameters

In [ ]:
params = {
    # Data
    'data_used': 'EM_N_fix_1500',
    'families_to_discard': [0, 5, 10, 20, 30],
    'seq_len': 100,
    'feature_dim': 7,
    
    # Training
    'epochs': 50,
    'val_split': 0.05,
    'batch_size': 32,
    'lr': 0.001,
    
    # Model
    'model_name': ['vae_conv5_legit', 'inception_time_wp_vae'],
    'latent_dim': [2, 6],
    'beta': [0.001, 0.2, 0.5, 1, 1.5, 2, 10],
    
    # Convergence
    'max_iter_convergence': 
    20,
    'input_seq_len_convergence': 1,
    
    # Evaluation
    'samples_to_generate': 100,
    'distance_metric': 'euclidean'
}

In [3]:
model_specific_params = {
    'vae_conv5_legit': {
        'dropout_rate': 0.2
    },
    'inception_time_wp_vae': {
        'n_filters': 32,
        'kernel_sizes': [3, 7, 13],
        'bottleneck_channels': 32
    }
}

Parameter Set

In [4]:
parameter_sets = generate_parameter_sets(params, model_specific_params)

In [5]:
len(parameter_sets)

140

In [ ]:
import random

random_parameter_sets = random.sample(parameter_sets, 20)

random_parameter_sets


[{'data_used': 'EM_N_fix_1500',
  'families_to_discard': 5,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val_split': 0.05,
  'batch_size': 32,
  'lr': 0.001,
  'model_name': 'inception_time_wp_vae',
  'latent_dim': 6,
  'max_iter_convergence': 20,
  'input_seq_len_convergence': 1,
  'samples_to_generate': 100,
  'distance_metric': 'euclidean',
  'model_kwargs': {'n_filters': 32,
   'kernel_sizes': [3, 7, 13],
   'bottleneck_channels': 32,
   'beta': 1.5}},
 {'data_used': 'EM_N_fix_1500',
  'families_to_discard': 20,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val_split': 0.05,
  'batch_size': 32,
  'lr': 0.001,
  'model_name': 'vae_conv5_legit',
  'latent_dim': 6,
  'max_iter_convergence': 20,
  'input_seq_len_convergence': 1,
  'samples_to_generate': 100,
  'distance_metric': 'euclidean',
  'model_kwargs': {'dropout_rate': 0.2, 'beta': 10}},
 {'data_used': 'EM_N_fix_1500',
  'families_to_discard': 5,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val

In [ ]:
notebook_to_execute = '03_01_generative_discovery.ipynb'
output_dir = "../experiments/03_01_generative_discovery"
checkpoint_file = '../experiments/experiment_checkpoint.json'
# notebook_to_output = '03_01_generative_discovery1.ipynb'

paralelize_notebook_experiment(
    random_parameter_sets,
    notebook_to_execute=notebook_to_execute,
    output_dir=output_dir,
    checkpoint_file=checkpoint_file,
    max_workers=5
)





You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


  | Name          | Type                      | Params | Mode 
--------------------------------------------------------------------
0 | encoder       | WPInceptionTimeVAEEncoder | 5.2 M  | train
1 | decoder       | WPInceptionTimeVAEDecoder | 5.7 M  | train
2 | sampling      | Sampling                  | 0      | train
3 | train_metrics | MetricCollection          | 0      | train
4 | val_metrics   | MetricCollection          | 0      | train
--------------------------------------------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
43.591    Total estimated model p

Executing:   0%|          | 0/160 [00:00<?, ?cell/s]





You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


  | Name          | Type                  | Params | Mode 
----------------------------------------------------------------
0 | encoder       | Conv5EncoderLegitTsgm | 3.4 M  | train
1 | decoder       | Conv5DecoderLegitTsgm | 3.4 M  | train
2 | sampling      | Sampling              | 0      | train
3 | train_metrics | MetricCollection      | 0      | train
4 | val_metrics   | MetricCollection      | 0      | train
----------------------------------------------------------------
6.7 M     Trainable params
0         Non-trainable params
6.7 M     Total params
26.862    Total estimated model params size (MB)
54        Module

Executing:   0%|          | 0/160 [00:00<?, ?cell/s]





You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


  | Name          | Type                      | Params | Mode 
--------------------------------------------------------------------
0 | encoder       | WPInceptionTimeVAEEncoder | 5.2 M  | train
1 | decoder       | WPInceptionTimeVAEDecoder | 5.7 M  | train
2 | sampling      | Sampling                  | 0      | train
3 | train_metrics | MetricCollection          | 0      | train
4 | val_metrics   | MetricCollection          | 0      | train
--------------------------------------------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
43.588    Total estimated model p